# Perform unconstrained runs

Input
- `dataout/fair_samples_forcing.nc`
    Uncertainty factors and coefficients of categorized forcing agents
    from `040_forcing_unc.ipynb`
- `dataout/parms_sample.nc`
    Sampled parameters (N=50,000) from `030_parms_sampling.ipynb`

Output
- `dataout/unconstrained_run.nc`
    Results from unconstrained runs over the historical period of 1750–2019

In [1]:
import sys
import yaml
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from tqdm.notebook import tqdm

In [2]:
# mce is available in https://github.com/tsutsui1872/mce 
sys.path.append('../mce')

In [3]:
from src.util import RetrieveGitHub, df2nc
from src.tlm import DriverMCE

In [4]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

## Forcing input with uncertainties

In [5]:
path = repo_ch7.retrieve('data_output/AR6_ERF_1750-2019.csv')
df_erf_hist = pd.read_csv(path).set_index('year')

[2024-07-21 16:19:28 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/AR6_ERF_1750-2019.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/AR6_ERF_1750-2019.csv on 2024-06-13


In [6]:
path = repo_ch7.retrieve('data_output/emissions_ceds_update_plus_bb.csv')
df_emis = pd.read_csv(path, index_col=0)

[2024-07-21 16:19:31 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/emissions_ceds_update_plus_bb.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/emissions_ceds_update_plus_bb.csv on 2024-06-13


In [7]:
nsize = 50000

ncf = Dataset('./dataout/fair_samples_forcing.nc')

df_erf_scale = pd.DataFrame({
    k: ncf.variables[f'scale_normals__{k}'][:nsize]
    for k in [
        'co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
        'contrails', 'bc_on_snow', 'land_use', 'volcanic', 'solar',
    ]
})

trend_solar = ncf.variables['trend_solar'][:nsize].filled()

aer_coeff = {
    k: ncf.variables[k][:nsize].filled()
    for k in ['beta_so2', 'beta_bc', 'beta_oc', 'beta_nh3', 'beta']
}
aer_coeff['aci_coeffs'] = np.array([
    ncf.variables['aci_coeffs__0'][:nsize].filled(),
    ncf.variables['aci_coeffs__1'][:nsize].filled(),
]).T

ncf.close()

## TLM parameter ensemble

In [8]:
ncf = Dataset('./dataout/parms_sample.nc')

method_order = [x for x in ncf.variables['Method'][:]]
names = ['q4x', 'q2x', 'lamg', 'cmix', 'cdeep', 'gamma_2l', 'eff', 'co2_beta']

df = [ncf.variables[k][:].filled(np.nan) for k in names]
df_parms_sample = pd.DataFrame(
    np.hstack(df),
    columns=pd.MultiIndex.from_product([names, method_order]),
)

ncf.close()

In [9]:
len(df_parms_sample) == nsize

True

## CO2 forcing scales

In [10]:
driver = DriverMCE()

In [11]:
q4x_ref = driver.q4x_ref
q4x_ref

8.259783657536742

In [12]:
df_scale_co2 = pd.concat([
    df_parms_sample.loc[:, ('q4x', method)].div(q4x_ref).rename(method)
    for method in ['ar6', 's21']
], axis=1)

In [13]:
df_scale_co2.apply(['mean', 'std'])

,ar6,s21
mean,0.943026,0.953481
std,0.128072,0.134166


## Ensemble member loop

In [14]:
cats = [
    'co2', 'ch4', 'n2o', 'other_wmghg', 'o3',
    'h2o_stratospheric', 'contrails', 'bc_on_snow', 'land_use',
    'volcanic', 'solar',
]
cats_aerosol = [
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
]
cats_nonco2 = cats[1:] + cats_aerosol

In [15]:
df_emis_a = df_emis.sub(df_emis.loc[1750])

In [16]:
with open('./src/variables.yml') as f1:
    var_atts = yaml.safe_load(f1)

In [17]:
variable_order = ['tg', 'ohc']

df = pd.DataFrame(
    0.,
    index=pd.Index([0], name='Member'),
    columns=pd.MultiIndex.from_product([
        variable_order,
        method_order,
        df_erf_hist.index,
    ], names=['Variable', 'Method', 'Year'])
)
path_out = './dataout/unconstrained_run.nc'
df2nc(path_out, df, var_atts)

ncf = Dataset(path_out, 'r+')

[2024-07-21 16:20:15 src.util] INFO:dataout/unconstrained_run.nc is created


In [18]:
# Requirements for the MCE forcing scheme

# object to get an equivalent CO2 concentration to a given forcing level
# based on the AR6 CO2 formula
driver.mk_interp_cco2()
interp_cco2 = driver.interp_cco2

# pre-industrial CO2 concentration
driver.forcing.parms.ccref = driver.cco2_pi_1750

In [19]:
for im in tqdm(range(nsize)):
    dfin = df_erf_hist.loc[:, cats].mul(df_erf_scale.loc[im, cats])

    dfin['solar'] += np.linspace(0, trend_solar[im], dfin.shape[0])

    dfin['aerosol-radiation_interactions'] = (
        df_emis_a['SO2'] * aer_coeff['beta_so2'][im] * 32./64.
        + df_emis_a['BC'] * aer_coeff['beta_bc'][im]
        + df_emis_a['OC'] * aer_coeff['beta_oc'][im]
        + df_emis_a['NH3'] * aer_coeff['beta_nh3'][im]
    )

    d1 = -aer_coeff['beta'][im] * np.log(
        1.
        + df_emis['SO2'].mul(32./64.) / aer_coeff['aci_coeffs'][im, 0]
        + df_emis[['BC', 'OC']].sum(axis=1) / aer_coeff['aci_coeffs'][im, 1]
    )
    dfin['aerosol-cloud_interactions'] = d1.sub(d1.loc[1750])

    df_parms = df_parms_sample.loc[im].unstack(0)

    ncf.variables['Member'][im] = im

    for jm, method in enumerate(method_order):
        p1 = df_parms.loc[method].dropna()
        driver.calib(p1)

        if method == 'ar6_orig':
            din_co2 = dfin['co2']
        elif method in ['ar6', 's21']:
            # Perturbed CO2 forcing is further scaled
            din_co2 = dfin['co2'] * df_scale_co2.loc[im, method]
        elif method == 'mce-2l':
            # Convert CO2 forcing to equivalent CO2 concentrations
            # and calculate forcing with the MCE CO2 scheme
            din_co2 = driver.forcing.c2erf(interp_cco2(dfin['co2'].values))
        else:
            raise ValueError(f'unexpected method {method}')

        din = din_co2 + dfin[cats_nonco2].sum(axis=1)
        dfout = driver.run(din)

        for vn, d1 in dfout.items():
            if vn not in variable_order:
                continue
            ncf.variables[vn][im, jm, :] = d1.values

ncf.close()

  0%|          | 0/50000 [00:00<?, ?it/s]